# Advent of Code 2022
## [Day 15: Beacon Exclusion Zone](https://adventofcode.com/2022/day/15)

#### Load Data

In [1]:
import aocd
input_data = aocd.get_data(year=2022, day=15).split("\n")
input_data[:5]

['Sensor at x=2389280, y=2368338: closest beacon is at x=2127703, y=2732666',
 'Sensor at x=1882900, y=3151610: closest beacon is at x=2127703, y=2732666',
 'Sensor at x=2480353, y=3555879: closest beacon is at x=2092670, y=3609041',
 'Sensor at x=93539, y=965767: closest beacon is at x=501559, y=361502',
 'Sensor at x=357769, y=2291291: closest beacon is at x=262473, y=2000000']

In [2]:
test_data = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""".split("\n")

### Part 1

In [3]:
import re
def parse_ints(line:str):
    return [int(i) for i in re.findall(r'(-?\d+)', line)]

parse_ints(test_data[0])

[2, 18, -2, 15]

In [4]:
from types import SimpleNamespace
from functools import cached_property

class Point(SimpleNamespace):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def dist(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)
    
    def tuning_frequency(self):
        return self.x * 4000000 + self.y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"

class Sensor(Point):
    def __init__(self, line):
        x, y, bx, by = parse_ints(line)
        self.x = x
        self.y = y
        self.beacon = Point(x=bx, y=by)
        
    def __repr__(self):
        return str(self.__dict__)
    
    @cached_property
    def radius(self):
        return self.dist(self.beacon)
        
s0 = Sensor(test_data[0])
s0, s0.radius

({'x': 2, 'y': 18, 'beacon': (-2, 15), 'radius': 7}, 7)

In [5]:
test_sensors = [Sensor(line) for line in test_data]
input_sensors = [Sensor(line) for line in input_data]

In [6]:
def can_contain_beacon(s, p):
    if p.x == s.x and p.y == s.y:
        return False
    if p == s.beacon:
        return True
    if s.dist(p) <= s.radius:
        return False
    return True

can_contain_beacon(s0, Point(-2,16))

False

In [7]:
def count_non_beacon(sensors, y=10):
    nearby_sensors = []
    for s in sensors:
        if s.dist(Point(s.x, y)) <= s.radius:
            nearby_sensors.append(s)
        
    min_x = min(s.x - s.radius for s in nearby_sensors)
    max_x = max(s.x + s.radius for s in nearby_sensors)
    count = 0
    print(min_x, max_x)
    for x in range(min_x, max_x+1):
        p = Point(x,y)
        for s in nearby_sensors:
            if not can_contain_beacon(s, p):
                count += 1
                break
    return count
    
count_non_beacon(test_sensors)

-8 28


26

#### Part 1 Answer
Consult the report from the sensors you just deployed.  
**In the row where y=2000000, how many positions cannot contain a beacon?**

In [8]:
%time count_non_beacon(input_sensors, 2000000)

-471146 5611670
CPU times: user 17 s, sys: 122 ms, total: 17.1 s
Wall time: 17.2 s


5809294

---

### Part 2

In [9]:
def cannot_contain_lost_beacon(s, p):
    return s.dist(p) <= s.radius

can_contain_beacon(s0, Point(-2,16))

False

In [10]:
def max_y_at_x(s, x):
    slack = s.radius - abs(s.x - x)
    y = s.y + slack
    return y

max_y_at_x(test_sensors[0], -2)

21

In [11]:
def find_beacon(sensors, max_x=20, max_y=20):
    x = 0
    while x <= max_x:
        if x % 100000 == 0:
            print(f"{x} / {max_x}")
        y = 0
        while y <= max_y:
            p = Point(x,y)
            valid = True
            for s in sensors:
                if s.dist(p) <= s.radius: 
                    valid = False
                    y = max_y_at_x(s, x)
                    break
            if valid:
                return p
            y += 1
        x += 1
    
find_beacon(test_sensors)

0 / 20


(14, 11)

In [12]:
%time find_beacon(input_sensors, max_x=4000000, max_y=4000000)

0 / 4000000
100000 / 4000000
200000 / 4000000
300000 / 4000000
400000 / 4000000
500000 / 4000000
600000 / 4000000
700000 / 4000000
800000 / 4000000
900000 / 4000000
1000000 / 4000000
1100000 / 4000000
1200000 / 4000000
1300000 / 4000000
1400000 / 4000000
1500000 / 4000000
1600000 / 4000000
1700000 / 4000000
1800000 / 4000000
1900000 / 4000000
2000000 / 4000000
2100000 / 4000000
2200000 / 4000000
2300000 / 4000000
2400000 / 4000000
2500000 / 4000000
2600000 / 4000000
CPU times: user 1min 15s, sys: 780 ms, total: 1min 16s
Wall time: 1min 17s


(2673432, 3308112)

#### Part 2 Answer
Find the only possible position for the distress beacon.  
**What is its tuning frequency?**

In [13]:
Point(2673432, 3308112).tuning_frequency()

10693731308112